<a href="https://colab.research.google.com/github/Jungin1020/Aiffel_exp/blob/main/2022_01_13_exp_4_7_songwriter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
import glob
import os, re
import numpy as np
import tensorflow as tf
import pathlib

txt_file_path = '/content/drive/MyDrive/aiffel/lyricist/data/lyrics/*'
txt_list = glob.glob(txt_file_path)
raw_corpus = []

for txt_file in txt_list:
  with open(txt_file,'r') as f:
    raw = f.read().splitlines()
    raw_corpus.extend(raw)

print('데이터 크기', len(raw_corpus))
print('Examples:\n',raw_corpus[:3])

Mounted at /content/drive
데이터 크기 187088
Examples:
 ['Looking for some education', 'Made my way into the night', 'All that bullshit conversation']


In [ ]:
for idx,sentence in enumerate(raw_corpus):
  if len(sentence) == 0: continue
  if idx > 9 : break

  print(sentence)

Looking for some education
Made my way into the night
All that bullshit conversation
Baby, can't you read the signs? I won't bore you with the details, baby
I don't even wanna waste your time
Let's just say that maybe
You could help me ease my mind
I ain't Mr. Right But if you're looking for fast love
If that's love in your eyes
It's more than enough


In [ ]:
def preprocess_sentence(sentence):
  sentence = sentence.lower().strip()
  sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
  sentence = re.sub(r'[" "]+', " ", sentence)
  sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
  sentence = sentence.strip()
  sentence = '<start> ' + sentence + ' <end>'
  return sentence


print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [ ]:
corpus = []
for sentence in raw_corpus:
  if len(sentence) == 0: continue

  preprocessed_sentence = preprocess_sentence(sentence)
  corpus.append(preprocessed_sentence)

corpus[:10]

['<start> looking for some education <end>',
 '<start> made my way into the night <end>',
 '<start> all that bullshit conversation <end>',
 '<start> baby , can t you read the signs ? i won t bore you with the details , baby <end>',
 '<start> i don t even wanna waste your time <end>',
 '<start> let s just say that maybe <end>',
 '<start> you could help me ease my mind <end>',
 '<start> i ain t mr . right but if you re looking for fast love <end>',
 '<start> if that s love in your eyes <end>',
 '<start> it s more than enough <end>']

In [ ]:
def tokenize(corpus):
  max_length = 15
  tokenizer = tf.keras.preprocessing.text.Tokenizer(
      num_words = 12000,
      filters = '',
      oov_token = '<unk>'
  )

  tokenizer.fit_on_texts(corpus)
  tensor = tokenizer.texts_to_sequences(corpus) #0 붙이기 전
  tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, maxlen=max_length, padding='post')

  print(tensor,tokenizer)
  return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)
tensor

[[  2 306  28 ...   0   0   0]
 [  2 221  13 ...   0   0   0]
 [  2  24  17 ...   0   0   0]
 ...
 [  2   5  33 ...   0   0   0]
 [  2   5  33 ...  13 169   3]
 [  2   5 372 ...   0   0   0]] <keras_preprocessing.text.Tokenizer object at 0x7f939ecf3d90>


array([[  2, 306,  28, ...,   0,   0,   0],
       [  2, 221,  13, ...,   0,   0,   0],
       [  2,  24,  17, ...,   0,   0,   0],
       ...,
       [  2,   5,  33, ...,   0,   0,   0],
       [  2,   5,  33, ...,  13, 169,   3],
       [  2,   5, 372, ...,   0,   0,   0]], dtype=int32)

# tf.keras.preprocessing.sequence.pad_sequences()
###maxlen 파라미터를 사용해 15 초과의 가사들을 잘라냈습니다.
###그래서 전체 토큰 개수에는 변화가 없었습니다.

In [ ]:
print(tensor[:5,:17])

[[   2  306   28   99 4814    3    0    0    0    0    0    0    0    0
     0]
 [   2  221   13   85  226    6  115    3    0    0    0    0    0    0
     0]
 [   2   24   17 1087 2353    3    0    0    0    0    0    0    0    0
     0]
 [ 629    6 2400   43    5  159   15 2762    7   31    6 5633    4   51
     3]
 [   2    5   37   15  163   86 1024   21   73    3    0    0    0    0
     0]]


In [ ]:
for idx in tokenizer.index_word:
  print(idx,':',tokenizer.index_word[idx])
  if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to


In [ ]:
scr_input = tensor[:,:-1]
tgt_input = tensor[:,1:]

print(scr_input[0])
print(tgt_input[0])

[   2  306   28   99 4814    3    0    0    0    0    0    0    0    0]
[ 306   28   99 4814    3    0    0    0    0    0    0    0    0    0]


In [ ]:
BUFFER_SIZE = len(scr_input)
BATCH_SIZE = 256
steps_per_epoch = len(scr_input)//BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1

dataset = tf.data.Dataset.from_tensor_slices((scr_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [ ]:
#from sklearn.model_selection import train_test_split
#enc_train, enc_val, dec_train, dec_val = train_test_split(scr_input,tgt_input,test_size=0.2,random_state=42)

In [ ]:
#print('Source Train:', enc_train.shape)
#print('target Train:', dec_train.shape)

In [ ]:
class TextGenerator(tf.keras.Model):
  def __init__(self, vocab_size, embedding_size, hidden_size):
    super().__init__()

    self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
    self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
    self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
    self.linear = tf.keras.layers.Dense(vocab_size)

  def call(self, x):
    out = self.embedding(x)
    out = self.rnn_1(out)
    out = self.rnn_2(out)
    out = self.linear(out)

    return out
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [ ]:
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)
model.compile(loss=loss,optimizer=optimizer)
model.fit(dataset, epochs=10)

Epoch 1/10
687/687 [==============================] - 116s 158ms/step - loss: 3.6073
Epoch 2/10
687/687 [==============================] - 116s 169ms/step - loss: 3.1352
Epoch 3/10
687/687 [==============================] - 118s 172ms/step - loss: 2.9420
Epoch 4/10
687/687 [==============================] - 119s 174ms/step - loss: 2.7870
Epoch 5/10
687/687 [==============================] - 120s 174ms/step - loss: 2.6506
Epoch 6/10
687/687 [==============================] - 120s 175ms/step - loss: 2.5271
Epoch 7/10
687/687 [==============================] - 120s 174ms/step - loss: 2.4127
Epoch 8/10
687/687 [==============================] - 120s 175ms/step - loss: 2.3059
Epoch 9/10
687/687 [==============================] - 120s 175ms/step - loss: 2.2046
Epoch 10/10
687/687 [==============================] - 120s 174ms/step - loss: 2.1070


In [ ]:
def generate_text(model,tokenizer, init_sentence='<start>',max_len=20):
  test_input = tokenizer.texts_to_sequences([init_sentence])
  test_tensor = tf.convert_to_tensor(test_input,dtype=tf.int64)
  end_token = tokenizer.word_index['<end>']

  while True:
    predict = model(test_tensor)
    predict_word = tf.argmax(tf.nn.softmax(predict,axis=-1), axis=-1)[:,-1]
    test_tensor = tf.concat([test_tensor,tf.expand_dims(predict_word,axis=0)], axis=-1)
    if predict_word.numpy()[0] == end_token: break
    if test_tensor.shape[1] >= max_len: break

  generated = ''
  for word_index in test_tensor[0].numpy():
    generated += tokenizer.index_word[word_index] + ' '

  return generated

In [ ]:
generate_text(model, tokenizer, init_sentence='<start> my')

'<start> my chick bad , badder than yours <end> '

### 매우 꼼꼼하고 이해하기 쉽게 설명되어 있어 자연어처리에 조금 가까워질 수 있었습니다.
### Embedding의 기능이 제일 신기했습니다.

### 마지막에 생성된 가사는 칸예나 에미넴 같은 힙합러의 데이터에서 나왔나 추측해 봅니다.